#Imports

In [1]:
!pip install datasets

In [2]:
!pip install giotto-tda

In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from numpy.random import default_rng
from datasets import load_dataset
rng = default_rng(42)  # Create a random number generator

from scipy.spatial.distance import pdist, squareform
from scipy.sparse import coo_matrix
from gtda.graphs import GraphGeodesicDistance
from gtda.homology import VietorisRipsPersistence, SparseRipsPersistence, FlagserPersistence
from igraph import Graph
from IPython.display import SVG, display
import requests

In [4]:
urls = [
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/scripts/directed_TDA_train.py",
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/scripts/baseline_train.py",
    "https://raw.githubusercontent.com/kzGarifullin/TDA-for-Transformers/main/scripts/directed_graphs_TDA.py"
]

for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, 'wb') as file:
        file.write(response.content)

print("Files downloaded successfully.")

Files downloaded successfully.


In [5]:
from directed_graphs_TDA import undirected_features, directed_features

# Dataset preparation

In [6]:
imdb_dataset = load_dataset("imdb")
texts = imdb_dataset["train"]["text"]
labels = imdb_dataset["train"]["label"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Load the pretrained En-BERT-base model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [8]:
max_length = 64 #128  # Maximum sequence length for BERT
full_token_ids = [tokenizer.encode(text, max_length=max_length, truncation=True) for text in texts]
full_token_ids = [ids + [0] * (max_length - len(ids)) for ids in full_token_ids]
full_token_ids_tensor = torch.tensor(full_token_ids)
full_labels_tensor = torch.tensor(labels)
full_dataset = TensorDataset(full_token_ids_tensor, full_labels_tensor)
batch_size = 32
full_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=False)

In [9]:
len(full_loader)

782

In [15]:
c = 0
for i in full_loader:
    print(i)
    c += 1
    if c > 5:
        break

[tensor([[  101,  1045, 12524,  ...,  1000,  1045,   102],
        [  101,  1000,  1045,  ...,  1005,  1056,   102],
        [  101,  2065,  2069,  ..., 11795,  3085,   102],
        ...,
        [  101,  1045,  2363,  ...,  1010,  2021,   102],
        [  101,  1045,  2031,  ...,  2018,  1996,   102],
        [  101,  1012,  1012,  ...,  1996,  2556,   102]]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])]
[tensor([[  101,  2017,  2031,  ...,  1998,  1045,   102],
        [  101,  1996, 11798,  ...,  7880,  1010,   102],
        [  101,  1037,  2450,  ...,  1997,  1996,   102],
        ...,
        [  101,  1045,  3984,  ...,  2002,  2003,   102],
        [  101,  1045,  2123,  ..., 17322,  2015,   102],
        [  101,  2023,  3185,  ...,  2151,  3425,   102]]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])]
[tensor([[  101,  4151, 10231,  ...,  2839, 

# Undirected features calculation

In [10]:
12*12*14

2016

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)
model = model.to(device)
print("device: ", device)
undirected_features = undirected_features(model, full_loader, device)

In [ ]:
import numpy as np
file_path = 'new_undirected_features_64.npy'
np.save(file_path, np.array(undirected_features))

In [ ]:
len(undirected_features)

25000

# Directed

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True)
model = model.to(device)
print("device: ", device)
directed_features = directed_features(model, full_loader, device)

In [ ]:
import numpy as np
file_path = 'new_directed_features_64.npy'
np.save(file_path, np.array(directed_features))

In [ ]:
len(directed_features)

25000